<a href="https://colab.research.google.com/github/thecodemancer/study-with-me/blob/main/python/food_classification_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![ceviche.avif](https://t1.rg.ltmcdn.com/es/posts/7/4/1/ceviche_peruano_18147_orig.jpg)

# Overview

This model is trained to recognize 2023 food dishes from images. It is based on MobileNet V1.

# Input
This model takes images as input.

Inputs are expected to be 3-channel RGB color images of size 224 x 224, scaled to [0, 1].

# Output
This model outputs to image_classifier.

image_classifier: A probability vector of dimension 2024, corresponding to a background class and one of 2023 food dishes in the labelmap.

# Model architecture
For details of the model architecture, see MobileNet V1.

# Suitable usecases
This model is trained to recognize 2023 food dishes. The training set includes entrees, side dishes, desserts, snacks, etc.

# Unsuitable usecases
Do not use this model to determine whether an object is edible or not.
This model is not suitable for predicting the ingredients of a food dish. Do not use this model to predict allergen or nutrition information.

# Known limitations
This model assumes that its input image contains a well-cropped food dish. If a non-food image is input to the model or if the dish is not well-cropped, the output of the model may be meaningless.
This model was trained on a dataset skewed toward North American foods.

In [179]:
# Import TF2 version
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [180]:
# Import other libraries
import numpy as np
import pandas as pd
import cv2
from skimage import io

In [181]:
mysterious_food_url = "https://t1.rg.ltmcdn.com/es/posts/7/4/1/ceviche_peruano_18147_orig.jpg" #ceviche
#mysterious_food_url = "https://imgmedia.buenazo.pe/1200x660/buenazo/original/2020/09/15/5f6108f5462f6040a51d2ac4.jpg" #arroz con pollo
#mysterious_food_url = "https://finde.latercera.com/wp-content/uploads/2020/07/empanadas-entrada-listo-.jpg" #empanada
input_shape = (224, 224)

In [182]:
image = np.asarray(io.imread(mysterious_food_url), dtype="float")
image

array([[[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       [[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       [[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       ...,

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 239., 245.],
        ...,
        [204., 214., 224.],
        [201., 211., 221.],
        [207., 217., 227.]],

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 239., 245.],
        ...,
        [205., 215., 225.],
        [205., 215., 225.],
        [208., 218., 228.]],

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 2

In [183]:
image = cv2.resize(image, dsize=input_shape, interpolation=cv2.INTER_CUBIC)
image

array([[[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.        , 231.        , 242.        ]],

       [[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.        , 232.1038208 , 239.7923584 ]],

       [[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.00000918, 232.00000918, 239.95032656]],

       ...,

      

In [184]:
# Scale values to [0, 1].
image = image / image.max()
image

array([[[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279213, 0.8532426 , 0.8938732 ]],

       [[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279213, 0.85731978, 0.88571886]],

       [[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279216, 0.85693633, 0.88630234]],

       ...,

       [[0.88092054, 0.87722685, 0.899389  ],
        [0.87504096, 0.87134727, 0.89350942],
        [0.87712874, 0

In [185]:
# The model expects an input of (?, 224, 224, 3).
images = np.expand_dims(image, 0)
images

array([[[[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279213, 0.8532426 , 0.8938732 ]],

        [[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279213, 0.85731978, 0.88571886]],

        [[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279216, 0.85693633, 0.88630234]],

        ...,

        [[0.88092054, 0.87722685, 0.899389  ],
         [0.87504096, 0.87134727, 0.89350942]

In [186]:
output = m(images)
print(output)

tf.Tensor(
[[1.3755466e-11 3.1406493e-08 4.5174867e-10 ... 2.1996420e-09
  8.2277585e-10 9.7932712e-12]], shape=(1, 2024), dtype=float32)


In [187]:
predicted_index = output.numpy().argmax()
print(predicted_index)

1923


In [188]:
df = pd.read_csv(labelmap_url)
print(df.head())

   id               name
0   0     __background__
1   1            Chaudin
2   2         Bambalouni
3   3            Ghoriba
4   4  Mango sticky rice


In [189]:
df.loc[1923]

id         1923
name    Ceviche
Name: 1923, dtype: object

In [190]:
df.loc[1394]

id                 1394
name    Arroz con pollo
Name: 1394, dtype: object

In [191]:
df.loc[470]

id           470
name    Empanada
Name: 470, dtype: object

In [192]:
classes = list(df["name"])
print(classes)

['__background__', 'Chaudin', 'Bambalouni', 'Ghoriba', 'Mango sticky rice', 'Jianbing', 'Aguachile', 'Carrozza', 'Miyan kuka', 'Efo riro', 'Ayam masak merah', 'Chiffon pie', '/g/11b8_rxx4d', 'Riso patate e cozze', 'Bazin', 'Black bottom pie', 'Palóc soup', 'Sailor sandwich', 'Tuwo shinkafa', 'Carne a la tampiqueña', 'Pastel azteca', 'Fujian red wine chicken', 'Boeber', 'Lady Baltimore cake', 'Yam khai dao', 'Texas Tommy', 'Har cheong gai', 'Kolokythopita', 'Karydopita', 'Rinflajš', 'Hainanese curry rice', 'Sonoran hot dog', '/g/11cfty6q3', 'Afghani burger', 'Teochew porridge', 'Minestra di ceci', 'Pastrami on rye', 'Roast beef sandwich', 'Chahan', 'Ekuru', 'Sciusceddu', 'Breakfast burrito', '/g/11dyjj24g', 'Sausage Stroganoff', 'Roti jala', 'Pirao', 'Casatiello', 'Khanom tan', 'Muamba chicken', 'Dobradinha', 'Bruckfleisch', 'Molote', 'Spongata', 'Funge', '/g/1212ghsj', 'Köttbullar', "Ka'ak", 'Papet vaudois', '/g/12148tdg', 'Prosciutto di Norcia', 'Malloreddus', '/g/1214g6v_', 'Pannenko

In [193]:
print("Prediction: ", classes[predicted_index])

Prediction:  Ceviche
